This is a simple illustration of a [Particle Swarm](https://en.wikipedia.org/wiki/Particle_swarm_optimization) trying to find the minimum of the [Rosenbrock function](https://en.wikipedia.org/wiki/Rosenbrock_function). Obviously the code is imperfect, but this should serve as a useful illustration.

Parameters can be set in the "PSO" function in the last cell of this notebook. Consider the following tasks:
- Higher search space dimensions (you may need more time in this case(
- change the population size (does a larger swarm imply shorter search time?>)
- Try out other parameter values
    - increase or decreast any of w, a1 or a2
    - change sign of w
- Implement a different goal function, e.g. the [Rastrigin function](https://en.wikipedia.org/wiki/Rastrigin_function). 
- Add an avarage for statistical evaluation (like wedid in the GA notebook)
- If you know python well, you can try to include an animation, e.g. to visualise search biasses 

In [28]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
import random

TOURNAMENT_SIZE = 5    # size of tournament for tournament selection
GENERATIONS = 250
PROB_MUTATION = 0.01
XO_RATE = 0.8

The following will be the goal ("fitness") function. Here it is to be minimised.

In [2]:
def rosenbrock(pos,dim):    # this serves as a goal function
                            # Defined by f(x,y) = (a-x)^2 + b(y-x^2)^2
                            # Using here: a = 1, b= 100, optimum 0 at (1,1)
        if dim==2:
            return ((1-pos[0])**2 + 100*(pos[1] - pos[0]**2)**2)
        elif dim==1:
            return (1-pos[0])**2 
        else: 
            ros=0;
            for i in range(dim-1):
                ros=ros+100*(pos[i+1] - pos[i]**2)**2 * (1-pos[i])**2 
            return ros

In [3]:
def f1(pos, dim):
    return sum([x**2 for x in pos])

def f2(pos, dim):
    return 10*dim + sum([(x**2 - 10*np.cos(2*np.pi*x)) for x in pos])

In [122]:
class Particle: # all the material that is relavant at the level of the individual particles
    
    def __init__(self, dim, minx, maxx,algo = 'r'):
        #np.random.seed(2021)
        self.position = np.random.uniform(low=minx, high=maxx, size=dim)
        self.velocity = np.random.uniform(low=-0.1, high=0.1, size=dim)
        self.best_particle_pos = self.position
        self.dim = dim
        self.algo = algo
 

        self.fitness = self.getfitness(self.position,dim)
        self.best_particle_fitness = self.fitness   # we couldd start with very large number here, 
                                                    #but the actual value is better in case we are lucky 
                
    def setPos(self, pos,algo = 'r'):
        self.position = pos
        self.fitness = self.getfitness(self.position,self.dim)
        
        if self.fitness<self.best_particle_fitness:     # to update the personal best both 
                                                        # position (for velocity update) and
                                                        # fitness (the new standard) are needed
                                                        # global best is update on swarm leven
            self.best_particle_fitness = self.fitness
            self.best_particle_pos = pos
      
            
    def updateVel(self, inertia, a1, a2, best_self_pos, best_swarm_pos):
                # Here we use the canonical version
                # V <- inertia*V + a1r1 (peronal_best - current_pos) + a2r2 (global_best - current_pos)
        cur_vel = self.velocity
        r1 = np.random.uniform(low=0, high=1, size = self.dim)
        r2 = np.random.uniform(low=0, high=1, size = self.dim)
        
        a1r1 = np.multiply(a1, r1)
        
        a2r2 = np.multiply(a2, r2)
        best_self_dif = np.subtract(best_self_pos, self.position)
        best_swarm_dif = np.subtract(best_swarm_pos, self.position)
                    # the next line is the main equation, namely the velocity update, 
                    # the velocities are added to the positions at swarm level 
        return inertia*cur_vel + np.multiply(a1r1, best_self_dif) + np.multiply(a2r2, best_swarm_dif)
    
    def updateVel_repulsive(self, inertia, a1, a2, a3, best_self_pos, best_swarm_pos, poses,idx):
        classic_terms = self.updateVel(inertia, a1, a2, best_self_pos, best_swarm_pos)
        r3 = np.random.uniform(low=0, high=1, size = self.dim)
        a3r3 = np.multiply(a3, r3)
        rep = 0
        for i in range(len(poses)):
            if not i == idx:
                dist = sum([x**2 for x in np.subtract(poses[i].position, self.position)])
                rep += np.subtract(poses[i].position, self.position)/(dist+np.random.uniform(low=0, high=0.2)**2)
                #print(poses[i].position)
        #print('term and pos:', np.multiply(a3r3, rep),self.position)
        #print(poses[0].position for i in range(len(poses)))
        return classic_terms + np.multiply(a3r3, rep)#/(len(poses)-1)
    
    def getfitness(self):
        if self.algo == 'f1':
            self.fitness = f1(self.position,self.dim)
        elif self.algo == 'f2':
            self.fitness = f2(self.position,self.dim)
        else:
            self.fitness = rosenbrock(self.position,self.dim)
        return self.fitness
    
    def partial(self, length):
        return (self.position[randint(0,self.dim-length):])[:length]
    
    def mutation(self):
        for i in range(self.dim):
            if random.random() < PROB_MUTATION:
                self.position[i] = random.uniform(-5.14,5.14)
                self.fitness = self.getfitness()
                
    def crossover(self, other):
        idx = self.dim
        if random.random() < XO_RATE:
            idx = random.randint(1,self.dim-2)
            print(idx)
        self.position = np.concatenate((self.position[:idx],other.position[idx:]),axis=0)
        self.fitness = self.getfitness()
        

In [123]:
class PSO: # all the material that is relavant at swarm leveel

    def __init__(self, w, a1, a2, dim=2, population_size=30, time_steps=1000, search_range=5.12,algo='r',rep=0,a3=0,printset = 10,show=1, swarms = []):

        # Here we use values that are (somewhat) known to be good
        # There are no "best" parameters (No Free Lunch), so try using different ones
        # There are several papers online which discuss various different tunings of a1 and a2
        # for different types of problems
        self.w = w # Inertia
        self.a1 = a2 # Attraction to personal best
        self.a2 = a2 # Attraction to global best
        self.a3 = a3
        self.printset = printset
        self.dim = dim
        self.rep = rep
        self.show = show
        if swarms:
            self.swarm = swarms
        else: 
            self.swarm = [Particle(dim,-search_range,search_range,algo) for i in range(population_size)]

        
        self.time_steps = time_steps
        #print('init')

        # Initialising global best, you can wait until the end of the first time step
        # but creating a random initial best and fitness which is very high will mean you
        # do not have to write an if statement for the one off case
        self.best_swarm_pos = np.random.uniform(low=-5.12, high=5.12, size=dim)
        self.best_swarm_fitness = 1e100


    def run(self):
        for t in range(self.time_steps):
            #print(t)
            for p in range(len(self.swarm)):
                particle = self.swarm[p]
                #print(particle.position)
                if self.rep == 0:
                    new_position = particle.position + particle.updateVel(self.w, self.a1, self.a2, particle.best_particle_pos, self.best_swarm_pos)
                else:
                    new_position = particle.position + particle.updateVel_repulsive(self.w, self.a1, self.a2, self.a3,
                                                                                    particle.best_particle_pos, self.best_swarm_pos,poses=self.swarm,idx=p)
                #print(new_position)
                if new_position@new_position > 1.0e+18: # The search will be terminated if the distance 
                                                        # of any particle from center is too large
                    print('Time:', t,'Best Pos:',self.best_swarm_pos,'Best Fit:',self.best_swarm_fitness)
                    raise SystemExit('Most likely divergent: Decrease parameter values')

                
                self.swarm[p].setPos(self.posWithinRange(new_position))
                new_fitness = self.swarm[p].getfitness(new_position,self.dim)

                if new_fitness < self.best_swarm_fitness:   # to update the global best both 
                                                            # position (for velocity update) and
                                                            # fitness (the new group norm) are needed
                    self.best_swarm_fitness = new_fitness
                    self.best_swarm_pos = new_position
            if self.show:
                if t % self.printset == 0: #we print only two components even it search space is high-dimensional
                    if self.dim == 2:
                        print("Time: %6d,  Best Fitness: %14.9f,  Best Pos: %9.4f,%9.4f" % (t,self.best_swarm_fitness,self.best_swarm_pos[0],self.best_swarm_pos[1]), end =" ") 
                    if self.dim == 3:
                        print("Time: %6d,  Best Fitness: %14.9f,  Best Pos: %7.4f,%7.4f,%7.4f\n" % (t,self.best_swarm_fitness,self.best_swarm_pos[0],self.best_swarm_pos[1],self.best_swarm_pos[2]), end =" ") 
                    elif self.dim>3: 
                        print('...')
                    else:
                        print('')

    def bestFitness(self):
        return self.best_swarm_fitness
                    
    def posWithinRange(self,pos):
        new_position = pos
        if new_position[0] > 5.12:
            new_position[0] = 5.12-np.random.uniform(low=0, high=1)
        elif new_position[0] < -5.12:
            new_position[0] = -5.12+np.random.uniform(low=0, high=1)

        if new_position[1] > 5.12:
            new_position[1] = 5.12-np.random.uniform(low=0, high=1)
        elif new_position[1] < -5.12:
            new_position[1] = -5.12+np.random.uniform(low=0, high=1)
        return new_position
              

Standard values are dim=2, w=0.7, a1=2.02, a2=2.02, population_size=30; but feel free to try others ones.

In [124]:
class PSOWithGA:

    def __init__(self, w, a1, a2, dim=2, population_size=30, time_steps=1000, search_range=5.12,algo='r',rep=0,a3=0,printset = 10,show=1, swarms = []):

        # Here we use values that are (somewhat) known to be good
        # There are no "best" parameters (No Free Lunch), so try using different ones
        # There are several papers online which discuss various different tunings of a1 and a2
        # for different types of problems
        self.w = w # Inertia
        self.a1 = a2 # Attraction to personal best
        self.a2 = a2 # Attraction to global best
        self.a3 = a3
        self.printset = printset
        self.dim = dim
        self.rep = rep
        self.show = show
        if swarms:
            self.swarm = swarms
        else: 
            self.swarm = [Particle(dim,-search_range,search_range,algo) for i in range(population_size)]

        
        self.time_steps = time_steps
        #print('init')

        # Initialising global best, you can wait until the end of the first time step
        # but creating a random initial best and fitness which is very high will mean you
        # do not have to write an if statement for the one off case
        self.best_swarm_pos = np.random.uniform(low=-5.12, high=5.12, size=dim)
        self.best_swarm_fitness = 1e100


    def run(self):
        for t in range(self.time_steps):
            #print(t)
            for p in range(len(self.swarm)):
                particle = self.swarm[p]
                #print(particle.position)
                if self.rep == 0:
                    new_position = particle.position + particle.updateVel(self.w, self.a1, self.a2, particle.best_particle_pos, self.best_swarm_pos)
                else:
                    new_position = particle.position + particle.updateVel_repulsive(self.w, self.a1, self.a2, self.a3,
                                                                                    particle.best_particle_pos, self.best_swarm_pos,poses=self.swarm,idx=p)
                #print(new_position)
                if new_position@new_position > 1.0e+18: # The search will be terminated if the distance 
                                                        # of any particle from center is too large
                    print('Time:', t,'Best Pos:',self.best_swarm_pos,'Best Fit:',self.best_swarm_fitness)
                    raise SystemExit('Most likely divergent: Decrease parameter values')

                
                self.swarm[p].setPos(self.posWithinRange(new_position))
                new_fitness = self.swarm[p].getfitness(new_position,self.dim)

                if new_fitness < self.best_swarm_fitness:   # to update the global best both 
                                                            # position (for velocity update) and
                                                            # fitness (the new group norm) are needed
                    self.best_swarm_fitness = new_fitness
                    self.best_swarm_pos = new_position
            if self.show:
                if t % self.printset == 0: #we print only two components even it search space is high-dimensional
                    if self.dim == 2:
                        print("Time: %6d,  Best Fitness: %14.9f,  Best Pos: %9.4f,%9.4f" % (t,self.best_swarm_fitness,self.best_swarm_pos[0],self.best_swarm_pos[1]), end =" ") 
                    if self.dim == 3:
                        print("Time: %6d,  Best Fitness: %14.9f,  Best Pos: %7.4f,%7.4f,%7.4f\n" % (t,self.best_swarm_fitness,self.best_swarm_pos[0],self.best_swarm_pos[1],self.best_swarm_pos[2]), end =" ") 
                    elif self.dim>3: 
                        print("Time: %6d,  Best Fitness: %14.9f,  Best Pos: %9.4f,%9.4f\n" % (t,self.best_swarm_fitness,self.best_swarm_pos[0],self.best_swarm_pos[1]), end =" ") 
                    else:
                        print('')

    def bestFitness(self):
        return self.best_swarm_fitness
                    
    def posWithinRange(self,pos):
        new_position = pos
        if new_position[0] > 5.12:
            new_position[0] = 5.12-np.random.uniform(low=0, high=1)
        elif new_position[0] < -5.12:
            new_position[0] = -5.12+np.random.uniform(low=0, high=1)

        if new_position[1] > 5.12:
            new_position[1] = 5.12-np.random.uniform(low=0, high=1)
        elif new_position[1] < -5.12:
            new_position[1] = -5.12+np.random.uniform(low=0, high=1)
        return new_position
              

In [ ]:
def GA(population):
    fitness = [p.fitness for p in population]
    for _ in range(GENERATIONS):        
        nextgen_population=[]
        
        for i in range(POP_SIZE):
            parent1 = selection(population, fitnesses)
            parent2 = selection(population, fitnesses)
            parent1.crossover(parent2)
            parent1.mutation()
            nextgen_population.append(parent1)
        population=nextgen_population

In [ ]:
def selection(population, fitnesses): # select one individual using tournament selection
    tournament = [randint(0, len(population)-1) for i in range(TOURNAMENT_SIZE)] # select tournament contenders
    tournament_fitnesses = [fitnesses[tournament[i]] for i in range(TOURNAMENT_SIZE)]
    return deepcopy(population[tournament[tournament_fitnesses.index(max(tournament_fitnesses))]]) 

In [129]:
population_size = 30
particles = []
for i in range(population_size):
     particles.append(Particle(10,-5.12,5.12,'f1'))
    

In [130]:
a = particles[0]
b = particles[1]

In [132]:
a.position

array([ 0.60762853, -0.04942355,  0.83569519,  0.36389006,  1.1452644 ,
        2.8620585 ,  2.67306361,  0.36207847,  4.13247195, -2.63432107])

In [133]:
b.position

array([ 0.34468868, -1.17309994,  3.62758857,  2.34083541,  1.0344212 ,
       -3.35971737,  1.12565168,  2.06450622, -0.64117224,  2.17699142])

In [136]:
a.crossover(b)

2


In [137]:
a.position

array([ 0.60762853, -0.04942355,  3.62758857,  2.34083541,  1.0344212 ,
       -3.35971737,  1.12565168,  2.06450622, -0.64117224,  2.17699142])

In [ ]:
PSO(dim=3, w=0.7, a1=2.02, a2=2.02, population_size=30, time_steps=1001, search_range=5.12,algo='f2',printset= 100).run()